In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set(rc={'figure.figsize':(11.7,8)})
sns.set_context("notebook", font_scale=2, rc={"lines.linewidth": 3})

In [2]:
def ComputeDensity(rho_0, cf, p,P_0):
    rho = rho_0 * np.exp(cf*(p-P_0))
    drhodp = cf * rho                        
    return rho, drhodp




def ComputePorosity(phi_0, cr, p,P_0):
    phi = phi_0 * np.exp(cr*(p-P_0))
    dphidp = cr * phi                            
    
    return phi, dphidp





def ComputeTransmissibility(Nx,dx,dA,Lambda,rho): 
    Tx = np.zeros(Nx+1)
    Lambda_rho = Lambda*rho
    for i in range(1,Nx):
        LambdaH_rho_H = 2*Lambda_rho[i-1]*Lambda_rho[i]/(Lambda_rho[i-1]+Lambda_rho[i]) 
        Tx[i] = LambdaH_rho_H*dA/(dx) #dV*LambdaH/(dx**2) 
    Tx[0] = Lambda_rho[0] *dA /(dx/2) 
    Tx[Nx] = Lambda_rho[Nx-1]*dA /(dx/2) 
    return Tx 



def ConstructA(Nx,Tx):
    A = np.zeros([Nx,Nx])
    for i in range(0,Nx): 
        # Left Neighbor 
        if i > 0:
            A[i,i-1] = -Tx[i] 
            A[i,i] = A[i,i] + Tx[i] 
        # Right Neighbor 
        if i < Nx-1:
            A[i,i+1] = -Tx[i+1] 
            A[i,i] = A[i,i] + Tx[i+1] 
    return A




def ComputeWellFluxes(Well,Lambda,dV,p,Nx):
    q = np.zeros(Nx)
    for w in range(0,len(Well["Pwell"])):
        # q = WI.Lambda[i].(Pwell-p[i]).dV
        i = Well["Cell"][w]
        q[i] = Well["WI"][w] * Lambda[i] * (Well["Pwell"][w] - p[i]) * dV 
    return q 

In [3]:
def Index2D(Nx, Ny, j, i):
    return Nx*j + i

def ComputeDensity2D(rho_0, cf, p,P_0):
    rho = rho_0 * np.exp(cf*(p-P_0))
    drhodp = cf * rho                        
    return rho, drhodp

def ComputePorosity2D(phi_0, cr, p,P_0):
    phi = phi_0 * np.exp(cr*(p-P_0))
    dphidp = cr * phi                            
    
    return phi, dphidp

def ComputeTransmissibility2D(Nx,Ny,dx,dy,dAx,dAy,Lambda,rho):
    Ty = np.zeros([Ny+1,Nx])
    Tx = np.zeros([Ny,Nx+1])
    
    Lambda_rho = Lambda * rho
    
    ## Tx-direction
    for i in range(1,Nx):       
        for j in range(0,Ny):   
            IW = Index2D(Nx,Ny,j,i-1)  
            IE = Index2D(Nx,Ny,j  ,i)  
            Lambda_rho_H = (2*Lambda_rho[IW]*Lambda_rho[IE])/(Lambda_rho[IW]+Lambda_rho[IE])
            Tx[j,i] = (dAx * Lambda_rho_H) / dx

    # East & West Boundry
    for j in range(0,Ny):
        # West Boundary
        i = 0
        I = Index2D(Nx,Ny,j,i)
        Lambda_rho_H = Lambda_rho[I]
        Tx[j,i] = dAx * Lambda_rho_H / (dx/2)
        # East Boundary
        i = Nx
        I = Index2D(Nx,Ny,j,i-1)
        Lambda_rho_H = Lambda_rho[I]
        Tx[j,i] = dAx * Lambda_rho_H / (dx/2)
        
    ## Ty-direction
    for i in range(0,Nx):      
        for j in range(1,Ny):  
            IS = Index2D(Nx,Ny,j-1,i)   
            IN = Index2D(Nx,Ny,j  ,i)  
            Lambda_rho_H = (2*Lambda_rho[IS]*Lambda_rho[IN])/(Lambda_rho[IS]+Lambda_rho[IN])
            Ty[j,i] = dAy * Lambda_rho_H / dx
    # For the boundary at north & south
    for i in range(0,Nx):
        # South Boundary
        j = 0
        I = Index2D(Ny,Nx,j,i)
        Lambda_rho_H = Lambda_rho[I]
        Ty[j,i] = dAy * Lambda_rho_H / (dy/2)
        # North Boundary
        j = Ny
        I = Index2D(Ny,Nx,j-1,i)
        Lambda_rho_H = Lambda_rho[I]
        Ty[j,i] = dAy * Lambda_rho_H / (dy/2)
    
    return Ty, Tx       
def ConstructA2D(Nx,Ny,Ty,Tx):
    A = np.zeros([ Ny*Nx , Ny*Nx ])
    
    for i in range(0,Nx):
        for j in range(0,Ny):
            IM = Index2D(Ny,Nx,j  ,i  )
            if i > 0:
                IW = Index2D(Ny,Nx,j,i-1)
                A[IW, IM] =  -Tx[j,i]
                A[IM, IM  ] = A[IM,IM] + Tx[j,i]
            if i < Nx-1:
                IE = Index2D(Ny,Nx,j,i+1)
                A[IE, IM] =  -Tx[j,i+1]
                A[IM, IM  ] = A[IM,IM] + Tx[j,i+1]
            if j > 0:
                IS = Index2D(Ny,Nx,j-1,i)
                A[IS, IM] =  -Ty[j,i]
                A[IM, IM  ] = A[IM,IM] + Ty[j,i]
            if j < Ny-1:
                IN = Index2D(Ny,Nx,j+1,i)
                A[IN, IM] =  -Ty[j+1,i]
                A[IM, IM  ] = A[IM,IM] + Ty[j+1,i]
    return A

def ComputeWellFluxes2D(Ny, Nx , Well,Lambda, dV, p):
    q = np.zeros(Ny*Nx)
    for w in range(len(Well["Pwell"])):
        j = Well["Cell"][0,w]
        i = Well["Cell"][1,w]
        I = Index2D(Nx,Ny,j,i)
        q[I] = Well["WI"][w] * Lambda[I] * (Well["Pwell"][w] - p[I]) * dV

    return q    

# 1D

In [ ]:
Lx = 1.0 # length of the system
Nx = 30  # number of cells
cf = 0.1 # compressibility of the fluid 
cr = 0.5 #compressibility of the rock
P_0 = 0  # initial pressure
rho_0 = 1 # initial density of the fluid 
phi_0 = 0.2 # initial rock porosity
# Well input
Well = {"Cell" : [0   , Nx-1],
        "WI"   : [1000, 1000],
        "Pwell": [2   ,    0]}
 
dx = Lx / Nx                       # The length of each grid cell
dA = 1.0                           # The cross-sectional area
dV = dx * dA                       # The volume of each grid cell
xc = np.linspace(dx/2, Lx-dx/2, Nx)
 
 
#Time settings

Nt = 2500                            # Number of time-steps
dt = 1e-5                         # Size of the time-step
 
 
# Mobility (lambda)
Lambda = np.ones(Nx)
# Lambda[13:16]=0.001 #hetro

## Initialization
p = P_0 * np.ones(Nx)                                # Initial pressure
rho,drhodp = ComputeDensity(rho_0,cf,p,P_0)          # Compute density and its derivative
phi,dphidp = ComputePorosity(phi_0,cr,p,P_0)         # Compute porosity and its derivative
Tx = ComputeTransmissibility(Nx,dx,dA,Lambda,rho)    # Compute transmissiblities
A = ConstructA(Nx,Tx)                                # Construct the coefficient matrix (A)
q = ComputeWellFluxes(Well,Lambda,dV,p,Nx)           # Compute volumetric well fluxes
 
 
# Plotting the pressure


 
 
##Entering the time loop

for t in range(0,Nt):
    Converged = 0
    Iter = 0
    rho_n = rho
    phi_n = phi
    
    # Entering iteration loop
    while Converged == 0:
        Iter = Iter + 1
        
        # Computing the compressibility (Psi)
        Psi = (dV/dt) * (rho*phi - rho_n*phi_n)
        
        ## Compute residual (q,Psi,Ap)
        Residual = rho*q - Psi - A@p
        
        ## Compute Jacobian (W,C,A)
        
        # W = d(rho.q)/dp = [dq/dp*rho] + [q*drho/dp]
        # For now, we neglect [q*drho/dp] !!!
        drho_q_dp = np.zeros(Nx)
        for w in range(0,len(Well["Pwell"])):
            i = Well["Cell"][w]
            drho_q_dp[i] = - rho[i] * Well["WI"][w] * Lambda[i] * dV
        W = np.diag(drho_q_dp)
        
        # C = d(Psi)/dp
        d_Psi_dp = (dV/dt) * ( drhodp*phi + rho*dphidp )
        C = np.diag(d_Psi_dp)
        
        Jacobian = W - C - A
        
        # Solve the linearized system (delta_p = J\-R)
        delta_p = np.linalg.solve(Jacobian,-Residual)
        
        # Update the pressure (p_new = p_old + delta_p)
        p = p + delta_p
        
        # Update the entire system
        rho,drhodp = ComputeDensity(rho_0,cf,p,P_0)
        phi,dphidp = ComputePorosity(phi_0,cr,p,P_0)
        Tx = ComputeTransmissibility(Nx,dx,dA,Lambda,rho)
        A = ConstructA(Nx,Tx)
        q = ComputeWellFluxes(Well,Lambda,dV,p,Nx)
        
        # Re-compute residual
        Psi = (dV/dt) * (rho*phi - rho_n*phi_n)
        Residual = rho*q - Psi - A@p
        
        
        # if convergence has happened -> out of the iteration loop
        # if not, go up and do all this again
        if np.linalg.norm(Residual) < 1e-6:
            Converged = 1
            
    # end of iteration loop
    
    # print('Time-step', t, ', # of iterations:', Iter)
    plt.plot(xc,p)
# end of time loop
plt.title('Pressure vs X') 
plt.ylabel('Pressure [Pa]')
plt.xlabel('X [m]')        
plt.show()

# 2D

In [9]:
# define the dimensions of the system
Lx = 1.0 # length of the system
dz = 1.0
Ly = 1.0
Ny = 10 # number of cells Y
Nx = 10  # number of cells X

cf = 1 # compressibility of the fluid 
cr = 1 #compressibility of the rock
P_0 = 0  # initial pressure
rho_0 = 1 # initial density of the fluid 
phi_0 = 0.2 # initial rock porosity

Well = {"Cell": np.array([[ 0, Ny-1 ],
                          [ 0, Nx-1 ]]),
        "WI": [1000, 1000],
        "Pwell": [2,0]}


#discretization in time and space
dy = Ly / Ny
dx = Lx / Nx                             # Length of each grid-cell

dAy = dx * dz
dAx = dy * dz                            # Cross-sectional area

dV = dy * dx * dz                        # Volume of each grid cell

xc = np.linspace( dx/2 , Lx-dx/2 , Nx   )  # Coordinates of cell centers   in x direction [m]
xi = np.linspace( 0    , Lx      , Nx+1 )  # Coordinates of cell interface in x direction [m]

yc = np.linspace( dy/2 , Ly-dy/2 , Ny   )  # Coordinates of cell centers   in y direction [m]
yi = np.linspace( 0    , Ly      , Ny+1 )  # Coordinates of cell interface in y direction [m]

# Mobility
# Lambda = np.ones((Nx,Ny))
# Lambda[4:6,:]=0.001
# Lambda = np.reshape(Lambda,(Nx*Ny)) 
Lambda = np.random.randint(low=0.1, high=10, size=(Nx*Ny))



Nt = 1000                            # Number of time-steps
dt = 1e-5                           # Size of the time-step
maxIter = 50   #maximum iteration


# Initialize the system 
p = P_0 * np.ones(Nx*Ny)               # initial pressure in all cells
rho,drhodp = ComputeDensity2D(rho_0,cf,p,P_0)
phi,dphidp = ComputePorosity2D(rho_0,cr,p,P_0)
Ty, Tx = ComputeTransmissibility2D(Nx,Ny,dx,dy,dAx,dAy,Lambda,rho)
A = ConstructA2D(Nx,Ny,Ty,Tx)
q = ComputeWellFluxes2D(Ny, Nx , Well,Lambda, dV, p)


 
 
##Entering the time loop

for t in range(0,Nt):
    Converged = 0
    Iter = 0
    rho_n = rho
    phi_n = phi
    
    # Entering iteration loop
    while Converged == 0 :#and Iter < maxIter:
        Iter = Iter + 1
        
        # Computing the compressibility (Psi)
        Psi = (dV/dt) * (rho*phi - rho_n*phi_n)
        
        ## Compute residual (q,Psi,Ap)
        Residual = rho*q - Psi - A@p
        
        ## Compute Jacobian (W,C,A)
        
        # W = d(rho.q)/dp = [dq/dp*rho] + [q*drho/dp]
        # For now, we neglect [q*drho/dp] !!!
        drho_q_dp = np.zeros(Nx*Ny)
        for w in range(len(Well["Pwell"])):
            j = Well["Cell"][0,w]
            i = Well["Cell"][1,w]
            I= Index2D(Nx, Ny, j, i)            
            drho_q_dp[I] = - rho[I] * Well["WI"][w] * Lambda[I] * dV
        W = np.diag(drho_q_dp)
        
        # C = d(Psi)/dp
        d_Psi_dp = (dV/dt) * ( drhodp*phi + rho*dphidp )
        C = np.diag(d_Psi_dp)
        
        Jacobian = W - C - A
        
        # Solve the linearized system (delta_p = J\-R)
        delta_p = np.linalg.solve(Jacobian,-Residual)
        # Update the pressure (p_new = p_old + delta_p)
        p = p + delta_p
        
        # Update the entire system
        rho,drhodp = ComputeDensity2D(rho_0,cf,p,P_0)
        phi,dphidp = ComputePorosity2D(phi_0,cr,p,P_0)
        Ty,Tx = ComputeTransmissibility2D(Nx,Ny,dx,dy,dAx,dAy,Lambda,rho)
        A = ConstructA2D(Nx,Ny,Ty,Tx)
        q = ComputeWellFluxes2D(Ny, Nx , Well,Lambda, dV, p)
        
        # Re-compute residual
        Psi = (dV/dt) * (rho*phi - rho_n*phi_n)
        Residual = rho*q - Psi - A@p
        
        
        # if convergence has happened -> out of the iteration loop
        # if not, go up and do all this again
        if np.linalg.norm(Residual) < 1e-6:
            Converged = 1
            
    # end of iteration loop
    
    # print('Time-step', t, ', # of iterations:', Iter)
# end of time loop
        
X,Y = np.meshgrid(xc,yc)
z_P = np.reshape(p,(Nx,Ny))
import plotly.graph_objects as go
fig = go.Figure(data=[go.Surface(z=z_P,x=X,y=Y,colorscale ='RdBu',reversescale=True)])
fig.update_layout(title='2D view of Pressure in the grid', autosize=True,
                   height=1000,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.update_xaxes(title_font_family="Arial")
fig.update_layout(
    font_family="Courier New",
    font_color="blue",
    title_font_family="Times New Roman",
    title_font_color="red",
    legend_title_font_color="green"
)
fig.update_xaxes(title_font_family="Arial")
fig.update_layout(width=900, height=800,autosize=True,font=dict(
        family="Courier New, monospace",
        size=15,
        color="black"
    ), title_x=0.5,    scene=dict(
        xaxis_title='X-coordinates [m]',
        yaxis_title='Y-coordinates [m]',
        zaxis_title='Pressure',))
fig.update_layout(
        scene = {
            "xaxis": {"nticks": 10},
            "zaxis": {"nticks": 5},
#             'camera_eye': {"x": 0, "y": -1, "z": 0.2},
            "aspectratio": {"x": 1, "y": 1, "z": 1}
        })
fig.show()

In [ ]:
Lambda

In [ ]:
Lambda = np.ones((Nx,Ny))
Lambda[2:3,:]=0.001
Lambda = np.reshape(Lambda,(Nx*Ny))

In [ ]:
Lambda